In [2]:
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'retina'


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
%pwd
%cd ..
%cd src
%pwd

/home/nas4_user/hojuncho/kyudan/AnomLLM/AnomLLM
/home/nas4_user/hojuncho/kyudan/AnomLLM/AnomLLM/src


/home/nas4_user/hojuncho/.cache/pypoetry/virtualenvs/anomllm-yItPHLz9-py3.10/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


'/home/nas4_user/hojuncho/kyudan/AnomLLM/AnomLLM/src'

In [4]:
#Please edit the BASE_DIR on your situation.

BASE_DIR = "/home/nas4_user/hojuncho/kyudan/AnomLLM/AnomLLM"

In [5]:
def make_localization_prompt(anomal_data):
        
    LOCALIZATION_PROMPT = f"""Detect the ranges of anomalies in this time series based on the x-axis coordinate, where the x-axis coordinate refers to the x value in (x, y) data. The patterns of anomalies are as follows.
    {anomal_data}\nList one by one, in JSON format. If there are no anomalies, answer with an empty list [].

    Output template:
    [{{"start": ..., "end": ...}}, {{"start": ..., "end": ...}}...]
    """
    return LOCALIZATION_PROMPT

In [6]:
def create_gt_data(anomaly_locations)->list :
    gt_data = []
    for i in anomaly_locations:
        anomal = i[0]
        gt_data.append([[int(x) for x in sublist] for sublist in anomal])
    return gt_data

In [7]:
#raw_text = raw_results['gpt-4o-mini (0shot-text)']['request']['messages'][0]['content'][0]['text']
#data_part = raw_text.split("\n\n")[0]
#data_part_list = data_part.
import numpy as np
from prompt import time_series_to_str

def create_idx_datalist(arr:np.ndarray)->list:
    """
    입력: array([[ 0.00000000e+00],
       [ 1.87381312e-01],
       [ 3.68124545e-01]]) 형태.
    출력: [(0, '0.0'),
            (1, '0.19'),
            (2, '0.37')] 형태.
    """
    idx_data_list  = []
    for i, data in enumerate(time_series_to_str(arr).split(' ')):
        idx_data_list.append((i,data))
    return idx_data_list

def create_idx_data(list_:list) -> str:
    """
    입력: 
    [(0, '0.0'),
    (1, '0.19'),
    (2, '0.37')] 

    출력:
    '(0,0.0), (1,0.19), (2,0.37)' 형태
    """
    formatted_items = []
    for item in list_:
        formatted_item = f"({item[0]},{item[1]})"
        formatted_items.append(formatted_item)
    idx_data = ', '.join(formatted_items)
    return idx_data

In [8]:
def create_anomal_series(series_list, gt):
    result_list = []
    result = []
    for diff_anom in gt:
        # ground truth index는 1부터 1000이라 아래와 같이 indexing 해야 out of range error 안 뜸.
        for i in range(diff_anom[0]-1,diff_anom[1]):
            result_list.append(series_list[i])
        result.append(result_list)
        result_list = []
    
    return result # 0.19 0.37 0.54 이런 형태.

In [9]:
def create_anomal_data_prompt(anomal_list):
    result = []
    
    for i, anomaly_group in enumerate(anomal_list, 1):
        result.append(f"anomaly data {i}:")
        
        values = [item[1] for item in anomaly_group]
        result.append(" ".join(values))
        
        if i < len(anomal_list):
            result.append("")
    
    return "\n".join(result)

In [10]:
from prompt import LIMIT_PROMPT


def create_localization_prompt_idx(
        idx,
        series,
        anomaly_locations,
):
    gt = create_gt_data(anomaly_locations)
    idx_data = create_idx_data(create_idx_datalist(series[idx]))
    #idx_list = create_idx_datalist(series[idx])
    anomal_text = create_anomal_series(create_idx_datalist(series[idx]), gt[idx]) #j번째 anomal
    
    messages = [
        {
            "role": "user",
            "content": idx_data 
                    + "\n\n" 
                    + LIMIT_PROMPT 
                    + make_localization_prompt(create_anomal_data_prompt(anomal_text))
        }
    ]
    
    return {
        "messages": messages,
        "temperature": 0.4,
        "stop": ["''''", " – –", "<|endoftext|>", "<|eot_id|>"]
    }


In [11]:
from openai_api import send_openai_request
import json
import time
import os
from loguru import logger
from data.synthetic import SyntheticDataset

def online_AD_with_retries(
    model_name: str,
    data_name: str,
    #request_func: callable,
    variant: str = "standard",
    num_retries: int = 4,
):
         
    # Initialize dictionary to store results
    results = {}

    # Configure logger
    log_fn = f"logs/synthetic/{data_name}/{model_name}/" + variant + ".log"
    logger.add(log_fn, format="{time} {level} {message}", level="INFO")
    results_dir = f'{BASE_DIR}/results/synthetic/{data_name}/{model_name}/'
    data_dir = f'{BASE_DIR}/data/synthetic/{data_name}/eval/'
    train_dir = f'{BASE_DIR}/data/synthetic/{data_name}/train/'
    jsonl_fn = os.path.join(results_dir, variant + '.jsonl')
    os.makedirs(results_dir, exist_ok=True)

    eval_dataset = SyntheticDataset(data_dir)
    eval_dataset.load()

    train_dataset = SyntheticDataset(train_dir)
    train_dataset.load()


    # prepare series and anomaly locations(gt)
    series = []
    anomaly_locations = []
    for idx in range(len(eval_dataset)): #(0부터 399)
        series.append(eval_dataset[idx][1].numpy()) # eval dataset의 문제 series.
        anomaly_locations.append(eval_dataset[idx][0].numpy()) # eval dataset의 gt가 될 것.
    

    # Load existing results if jsonl file exists
    if os.path.exists(jsonl_fn):
        with open(jsonl_fn, 'r') as f:
            for line in f:
                entry = json.loads(line.strip())
                results[entry['custom_id']] = entry["response"]

    # Loop over image files
    for i in range(0, len(eval_dataset)):
        custom_id = f"{data_name}_{model_name}_{variant}_{str(i).zfill(5)}"
        gt = create_gt_data(anomaly_locations)

        # If there are no anomaly, response is forced to "```json\n[]\n```\n"
        # and others (custom_id, request) are same.
        if len(gt[i]) == 0:
            logger.info(f"Setting empty list response for {custom_id} as it has no anomaly locations")
            request = create_localization_prompt_idx(i, series, anomaly_locations)
            response = "```json\n[]\n```\n"
            # Write the result to jsonl
            with open(jsonl_fn, 'a') as f:
                json.dump({'custom_id': custom_id, 'request': request, 'response': response}, f)
                f.write('\n')
            results[custom_id] = response
            continue

        # Skip already processed files
        if custom_id in results:
            continue
        
        # Perform anomaly detection with exponential backoff
        for attempt in range(num_retries):
            try:
                request = create_localization_prompt_idx(i, series, anomaly_locations)
                response = send_openai_request(request, model_name)
                # Write the result to jsonl
                with open(jsonl_fn, 'a') as f:
                    json.dump({'custom_id': custom_id, 'request': request, 'response': response}, f)
                    f.write('\n')
                # If successful, break the retry loop
                break
            except Exception as e:
                if "503" in str(e):  # Server not up yet, sleep until the server is up again
                    while True:
                        logger.debug("503 error, sleep 30 seconds")
                        time.sleep(30)
                        try:
                            response = send_openai_request(request, model_name)
                            break
                        except Exception as e:
                            if "503" not in str(e):
                                break
                else:
                    logger.error(e)
                    # If an exception occurs, wait and then retry
                    wait_time = 2 ** (attempt + 3)
                    logger.debug(f"Attempt {attempt + 1} failed. Waiting for {wait_time} seconds before retrying...")
                    time.sleep(wait_time)
                    continue
        else:
            logger.error(f"Failed to process {custom_id} after {num_retries} attempts")

In [ ]:
online_AD_with_retries('gemini-1.5-flash', 'trend', 'localization (indexData)')